In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

# Import data
train = pd.read_csv("train.csv", delimiter = ",")
test = pd.read_csv("test.csv", delimiter = ",")

In [2]:
# Prepare train data
train_nona = train.dropna()
x_train = train_nona['Reviews']
y_train = train_nona['Rating']

# Prepare test data
x_test = test['Reviews']
x_test = x_test.fillna('.')

# Tokenizing reviews and creating feature dict
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [3]:
# Training the model
rfc = RandomForestClassifier(n_estimators = 300, random_state = 111, n_jobs = -1, verbose = True)
rfc.fit(x_train, y_train)

# Testing the model
predictions = rfc.predict(x_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 16.0min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    8.3s
[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:   13.8s finished


In [4]:
# Exporting results
test['Rating'] = predictions
results = test[['Id','Rating']]
results.to_csv("results.csv", index = False)